In [0]:
%fs
ls /

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/cluster-logs/,cluster-logs/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/dbacademy/,dbacademy/,0
dbfs:/dbfs/,dbfs/,0
dbfs:/delta/,delta/,0
dbfs:/foo.csv/,foo.csv/,0
dbfs:/mnt/,mnt/,0


In [0]:
%fs
ls dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/

path,name,size
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_inventory_snap.csv,store_inventory_snap.csv,491
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_inventory_txn.csv,store_inventory_txn.csv,461
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_order_transaction.csv,store_order_transaction.csv,1056
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_product_snap.csv,store_product_snap.csv,385
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/titanic-1.csv,titanic-1.csv,44225
dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/titanic.csv,titanic.csv,44225


In [0]:
%sql
DROP TABLE IF EXISTS store_inventory_txn;

--CREATE TABLE store_inventory_txn USING CSV OPTIONS (path "dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_inventory_txn.csv", header "true")
CREATE TABLE store_inventory_txn USING CSV OPTIONS (path "dbfs:/FileStore/shared_uploads/wchen@nextpathway.com/store_inventory_txn_2.csv", header "true")

In [0]:
%sql
SELECT material_key, inventory_transaction_type, 
  inventory_transaction_quantity, inventory_transaction_dollar_amount, capture_ts,
  store_key, date(capture_ts) as capture_date FROM store_inventory_txn

material_key,inventory_transaction_type,inventory_transaction_quantity,inventory_transaction_dollar_amount,capture_ts,store_key,capture_date
mtrl_01,add,100,200.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_02,add,50,100.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_03,remove,10,50.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_21,remove,20,80.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_22,add,200,800.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_23,remove,100,250.00,2021-08-08 08:00:00,store_0001,2021-08-08
mtrl_01,add,100,200.00,2021-08-08 08:00:00,store_0002,2021-08-08
mtrl_02,add,50,100.00,2021-08-08 08:00:00,store_0002,2021-08-08
mtrl_03,remove,10,50.00,2021-08-08 08:00:00,store_0002,2021-08-08
mtrl_21,remove,20,80.00,2021-08-08 08:00:00,store_0002,2021-08-08


In [0]:
%sql
--DROP TABLE IF EXISTS edw_store_inventory_txn;
CREATE TABLE edw_store_inventory_txn
  USING PARQUET
  PARTITIONED BY (store_key, capture_date)
--  CLUSTERED BY (length) INTO 8 buckets
  AS SELECT material_key, inventory_transaction_type, 
  inventory_transaction_quantity, inventory_transaction_dollar_amount, capture_ts,
  store_key, date(capture_ts) as capture_date FROM store_inventory_txn;

In [0]:
%sql
--desc formatted edw_store_inventory_txn;
--show partitions edw_store_inventory_txn;
--select * from edw_store_inventory_txn where capture_date = '2021-08-07';
select capture_ts,  date(cast(capture_ts as timestamp)) as capture_date
from store_inventory_txn;

capture_ts,capture_date
2021-08-07 08:00:00,2021-08-07
2021-08-07 08:00:00,2021-08-07
2021-08-07 08:00:00,2021-08-07
2021-08-07 08:00:00,2021-08-07
2021-08-07 08:00:00,2021-08-07
2021-08-07 08:00:00,2021-08-07


In [0]:
%sql
DROP TABLE IF EXISTS edw_store_inventory_txn;
CREATE TABLE edw_store_inventory_txn
( material_key string, 
  inventory_transaction_type string, 
  inventory_transaction_quantity int, 
  inventory_transaction_dollar_amount decimal, 
  capture_ts timestamp,
  store_key string,
  capture_date date
  )
  USING PARQUET
  PARTITIONED BY (store_key, capture_date);

In [0]:
%sql
insert into edw_store_inventory_txn partition(store_key, capture_date)
select material_key, inventory_transaction_type, 
    cast(inventory_transaction_quantity as int) inventory_transaction_quantity,
    cast(inventory_transaction_dollar_amount as decimal) inventory_transaction_dollar_amount, 
    cast(capture_ts as timestamp) capture_ts, 
    store_key, 
    date(cast(capture_ts as timestamp)) as capture_date 
from store_inventory_txn;

In [0]:
%sql
-- set hive.exec.dynamic.partition
-- set hive.exec.dynamic.partition.mode
show partitions edw_store_inventory_txn;
--select capture_date, count(*) from edw_store_inventory_txn group by capture_date;

partition
store_key=store_0001/capture_date=2021-08-06
store_key=store_0001/capture_date=2021-08-07
store_key=store_0001/capture_date=2021-08-08
store_key=store_0002/capture_date=2021-08-08
store_key=store_0003/capture_date=2021-08-09


In [0]:
%sql
--select order_number,order_type, order_line_number, line_net_dollar_amt, capture_ts, store_key from edw_store_order_txn;
select order_number,capture_ts, store_key, sum(line_net_dollar_amt) as sum_per_order
from edw_store_order_txn 
group by order_number,capture_ts, store_key;

order_number,capture_ts,store_key,sum_per_order
1001,2021-08-07T08:05:00.000+0000,store_0001,338
1002,2021-08-07T08:05:00.000+0000,store_0001,278
1201,2021-08-08T08:05:00.000+0000,store_0002,80
1202,2021-08-08T08:05:00.000+0000,store_0001,139
1201,2021-08-08T08:05:00.000+0000,store_0001,169
1202,2021-08-08T08:05:00.000+0000,store_0002,139
1003,2021-08-07T10:05:00.000+0000,store_9999,338
1203,2021-08-08T10:05:00.000+0000,store_9999,169


In [0]:
%sql
with osum as(
select order_number,capture_ts, store_key, sum(line_net_dollar_amt) as sum_per_order
from edw_store_order_txn 
group by order_number,capture_ts, store_key)

select 
osum.order_number, osum.capture_ts, osum.store_key, osum.sum_per_order,avg(osum.sum_per_order)
over w as moving_avg_3_ord_per_store
from osum
window w as (partition by osum.store_key order by osum.capture_ts 
rows between 2 preceding and current row)
;

order_number,capture_ts,store_key,sum_per_order,moving_avg_3_ord_per_store
1001,2021-08-07T08:05:00.000+0000,store_0001,338,338.0
1002,2021-08-07T08:05:00.000+0000,store_0001,278,308.0
1202,2021-08-08T08:05:00.000+0000,store_0001,139,251.6667
1201,2021-08-08T08:05:00.000+0000,store_0001,169,195.3333
1201,2021-08-08T08:05:00.000+0000,store_0002,80,80.0
1202,2021-08-08T08:05:00.000+0000,store_0002,139,109.5
1003,2021-08-07T10:05:00.000+0000,store_9999,338,338.0
1203,2021-08-08T10:05:00.000+0000,store_9999,169,253.5
